In [1]:
import os
from glob import glob
import arcpy
from arcpy import env

In [2]:
## step3: IDW interpolation ##

wd = ''

for depth in ['surface', 'mesopelagic', 'bathypelagic', 'abyssopelagic']:
    masktif = '%s/topo/%s.tif' % (wd, depth)
    for scen in ['ssp126', 'ssp245', 'ssp585']:
        for yr in range(2020,2101):
            env.workspace = '%s/step2_cmip6_expand/%s/%s/%s' % (wd, depth, scen, yr)
            os.chdir('%s/step3_cmip6_idw/%s/%s/%s' % (wd, depth, scen, yr))
            files = sorted(glob("*shp"))
            for f in files:
                name = f.split('.shp')[0]
                to_path = "%s/step3_cmip6_idw/%s/%s/%s/%s.tif" % (wd, depth, scen, yr, name)
                with arcpy.EnvManager(mask=masktif, extent="-180 -90 180 90"):
                    arcpy.gp.Idw_sa(f, "value", to_path, "1", "3", "VARIABLE 12", "")

            print("%s - %s - %s - complete" % (depth, scen, yr)) 

In [3]:
## step5: smooth data by moving-window averaging ##

for depth in ['surface', 'mesopelagic', 'bathypelagic', 'abyssopelagic']:
    masktif = '%s/topo/%s.tif' % (wd, depth)
    for scen in ['ssp126', 'ssp245', 'ssp585']:
        from_path = '%s/step4_cmip6_mdmean/%s/%s' % (wd, depth, scen)
        to_path = '%s/step5_cmip6_smooth/%s/%s' % (wd, depth, scen)
        
        os.chdir(from_path)
        files = sorted(glob("*tif"))
        for f in files:
            with arcpy.EnvManager(mask=masktif, extent="-180 -90 180 90"):
                out_raster = arcpy.ia.FocalStatistics(f, "Rectangle 3 3 CELL", "MEAN"); 
                out_raster.save("%s/%s" % (to_path, f))
            print("%s finish" % f)
        print("======== %s - %s: complete ========" % (depth, scen))